<a href="https://colab.research.google.com/github/rosyanas/A-B_Testing_Project_Pacmann-Photos_In_Ads/blob/main/Project_Experimental_Design_and_A_B_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# load data
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import data
filename = "/content/drive/MyDrive/Documentation/Pacmann/Project Experimental Design and A B Testing/AB_New_Paysystem V2.csv"
data = pd.read_csv(filename,
                           sep = ",")

# Tampilkan data
data.head()

,internal_time,milliseconds,action,user_id,user_country,ui_version,experiment_group,num_event,invoice_created
0,12/15/2021 4:10,1.639530e+12,open-list,2655,IL,mobile,B,1,1
1,12/15/2021 4:14,1.639530e+12,open-list,2655,IL,mobile,B,1,1
2,12/15/2021 4:15,1.639530e+12,open-list,2655,IL,mobile,B,1,0
3,12/15/2021 4:34,1.639530e+12,open-list,2655,IL,mobile,B,1,1
4,12/20/2021 8:27,1.639980e+12,open-list,2655,IL,mobile,B,1,1


In [4]:
# Get data shape
data.shape

(562343, 9)

**There are 9 column**
- `internal_time` : time click the event
- `milliseconds` : milisecound from `internal_time`
- `action` :
            open-list as click Ads and convert to website view
            create-invoice as click to check out the product
- `user_id` : unique user identificator
- `user_contry` : user country
- `ui_version` : where you find the Ads; mobile or desktop
- `experiment_group` : experiment group (Ads with smiling photos
 and Ads with exotic photos)
- `num_event` : number of event that defined in analytics flatform
- `invoice_created` : the goals that we set in the event (After check out and invoice status not cancel)

because we want to compare which photos in Ads that will convert user to purchase our fashion product. So the `action` that we'll use is `open-list`

## **<font color = 'blue'>Data Cleaning**

### **1. Filter `open-list` in colum action**

In [5]:
# Select only open-list action

data = data[data['action']=='open-list'].reset_index(drop=True)
data.head()

,internal_time,milliseconds,action,user_id,user_country,ui_version,experiment_group,num_event,invoice_created
0,12/15/2021 4:10,1.639530e+12,open-list,2655,IL,mobile,B,1,1
1,12/15/2021 4:14,1.639530e+12,open-list,2655,IL,mobile,B,1,1
2,12/15/2021 4:15,1.639530e+12,open-list,2655,IL,mobile,B,1,0
3,12/15/2021 4:34,1.639530e+12,open-list,2655,IL,mobile,B,1,1
4,12/20/2021 8:27,1.639980e+12,open-list,2655,IL,mobile,B,1,1


In [6]:
# Get data shape
data.shape

(419500, 9)

### **2. Checking data type per column**

In [7]:
# Get data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419500 entries, 0 to 419499
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   internal_time     419500 non-null  object 
 1   milliseconds      419500 non-null  float64
 2   action            419500 non-null  object 
 3   user_id           419500 non-null  int64  
 4   user_country      419500 non-null  object 
 5   ui_version        419500 non-null  object 
 6   experiment_group  419500 non-null  object 
 7   num_event         419500 non-null  int64  
 8   invoice_created   419500 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 28.8+ MB


In [8]:
# Change datatype `user_id` to object / string
data['user_id'] = data['user_id'].astype(str)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419500 entries, 0 to 419499
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   internal_time     419500 non-null  object 
 1   milliseconds      419500 non-null  float64
 2   action            419500 non-null  object 
 3   user_id           419500 non-null  object 
 4   user_country      419500 non-null  object 
 5   ui_version        419500 non-null  object 
 6   experiment_group  419500 non-null  object 
 7   num_event         419500 non-null  int64  
 8   invoice_created   419500 non-null  int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 28.8+ MB


### **3. Remove duplicate data**

In [9]:
# Create id from user_id and internal_time
data['id'] = data['internal_time'] + data['user_id']

# Checking duplicate data
data.duplicated(['id']).sum()

62631

In [10]:
#menampilkan data yang duplikat

ids = data['id']
data_dup=data[ids.isin(ids[ids.duplicated()])].sort_values("id")
data_dup

,internal_time,milliseconds,action,user_id,user_country,ui_version,experiment_group,num_event,invoice_created,id
127451,12/10/2021 12:27,1.639130e+12,open-list,1497196296,KR,desktop,A,1,0,12/10/2021 12:271497196296
127450,12/10/2021 12:27,1.639130e+12,open-list,1497196296,KR,desktop,A,1,0,12/10/2021 12:271497196296
89596,12/10/2021 12:30,1.639130e+12,open-list,1205250961,KR,desktop,A,1,1,12/10/2021 12:301205250961
89595,12/10/2021 12:30,1.639130e+12,open-list,1205250961,KR,desktop,A,1,0,12/10/2021 12:301205250961
115568,12/10/2021 12:30,1.639130e+12,open-list,1415289421,KR,desktop,A,1,0,12/10/2021 12:301415289421
...,...,...,...,...,...,...,...,...,...,...
127355,12/24/2021 9:59,1.640330e+12,open-list,1496846765,KR,mobile,A,1,0,12/24/2021 9:591496846765
168443,12/24/2021 9:59,1.640330e+12,open-list,1766368991,KR,desktop,B,1,0,12/24/2021 9:591766368991
168444,12/24/2021 9:59,1.640330e+12,open-list,1766368991,KR,desktop,B,1,0,12/24/2021 9:591766368991
188135,12/24/2021 9:59,1.640330e+12,open-list,1899045738,KR,desktop,B,1,0,12/24/2021 9:591899045738


In [11]:
# Dropping duplicates data
data.drop_duplicates(subset='id',keep='first',inplace=True)
data.shape

(356869, 10)

### **4. Checking missing value**

In [12]:
# Checking missing value
data.isna().sum()

internal_time       0
milliseconds        0
action              0
user_id             0
user_country        0
ui_version          0
experiment_group    0
num_event           0
invoice_created     0
id                  0
dtype: int64

## **<font color = 'blue'>Data Analysis**

### **1. Analysis Conversion Rate**

Before doing the chi-square test, let's fill in the following table to make it easier to detect the presence of SRM.


<center>

|Group|# id|# id convert to create invoice|CR|
|:--|:--:|:--:|:--:|
|A|-|-|-|-|
|B|-|-|-|-|

</center>

* Number of `id` is the sample size in the dataset per `experiment_group`
* Number of `id` convert to create invoice is the sample size in data set per `experiment_group` where value in `invoice_created` is `1`
* CR is the percentage of # user convert devided by # user

In [13]:
data_a = data[data["experiment_group"] == 'A']

data_a.head()

,internal_time,milliseconds,action,user_id,user_country,ui_version,experiment_group,num_event,invoice_created,id
6,12/19/2021 17:01,1.639920e+12,open-list,33026,KR,desktop,A,1,1,12/19/2021 17:0133026
7,12/15/2021 17:20,1.639580e+12,open-list,69463,HU,desktop,A,1,1,12/15/2021 17:2069463
11,12/17/2021 16:01,1.639750e+12,open-list,162121,BR,desktop,A,1,0,12/17/2021 16:01162121
12,12/17/2021 16:36,1.639750e+12,open-list,162121,BR,desktop,A,1,0,12/17/2021 16:36162121
13,12/17/2021 16:59,1.639750e+12,open-list,162121,BR,desktop,A,1,0,12/17/2021 16:59162121


In [14]:
data_b = data[data["experiment_group"] == 'B']

data_b.head()

,internal_time,milliseconds,action,user_id,user_country,ui_version,experiment_group,num_event,invoice_created,id
0,12/15/2021 4:10,1.639530e+12,open-list,2655,IL,mobile,B,1,1,12/15/2021 4:102655
1,12/15/2021 4:14,1.639530e+12,open-list,2655,IL,mobile,B,1,1,12/15/2021 4:142655
2,12/15/2021 4:15,1.639530e+12,open-list,2655,IL,mobile,B,1,0,12/15/2021 4:152655
3,12/15/2021 4:34,1.639530e+12,open-list,2655,IL,mobile,B,1,1,12/15/2021 4:342655
4,12/20/2021 8:27,1.639980e+12,open-list,2655,IL,mobile,B,1,1,12/20/2021 8:272655


In [15]:
# Number of users in the each group
n_a = data_a.shape[0]
n_b = data_b.shape[0]

print(f"Jumlah sample pada expreriment group A ={n_a}")

print(f"Jumlah sample pada expreriment group B ={n_b}")

Jumlah sample pada expreriment group A =178145
Jumlah sample pada expreriment group B =178724


In [16]:
# Number of users convert in the each group
n_a_convert = data_a[data_a["invoice_created"] == 1].shape[0]
n_b_convert = data_b[data_b["invoice_created"] == 1].shape[0]

print(f"Jumlah sample pada expreriment group A yg convert membuat invoice ={n_a_convert}")

print(f"Jumlah sample pada expreriment group B yg convert membuat invoice ={n_b_convert}")

Jumlah sample pada expreriment group A yg convert membuat invoice =65499
Jumlah sample pada expreriment group B yg convert membuat invoice =65672


In [17]:
# Percentage in each group
persen_a = n_a_convert/n_a * 100
persen_b = n_b_convert/n_b * 100

print(f"% A : {persen_a:.2f}%")
print(f"% B : {persen_b:.2f}%")

% A : 36.77%
% B : 36.74%


Dari hasil analisis conversion rate didapat :

<center>

|Group|# id|# id convert to create invoice|CR|
|:--|:--:|:--:|:--:|
|A|178,145|65,499|36.77%|
|B|178,724|65,672|36.74%|

</center>

Karena conversion rate dari `experiment_group` A dan B tidak berbeda jauh maka bisa kita simpulkan bahwa Ads menggunakan foto tersenyum ataupun exotic tidak berpengaruh.

Tapi akan kita coba lakukan Uji Asumsi sebagai validasi dan mencari saran selanjutnya.

### **2. Analysis Uji Asumsi**

Karena kita ingin menguji apakah lebih banyak user **convert** menjadi `invoice_created` dengan menggunakan foto tersenyum di dalam Ads daripada foto exotic.

Asumsikan : proporsi user convert menggunakan foto tersenyum adalah $p_1$ dan proposi user convert menggunakan foto exotic adalah $p_2$

>$H_0$ : $p_1$=$p_2$ <br>
$H_1$ : $p_1$ > $p_2$

dengan ignifican level sebesar 0.05<br>
>$\alpha=0.05$

In [18]:
#install for uji asumsi

import statsmodels.stats.api as sms
from scipy.stats import shapiro, levene, mannwhitneyu

import warnings
warnings.filterwarnings('ignore')

**a. Uji Normalitas**

Uji normalitas ialah untuk melihat apakah ada nilai residu normal atau tidak. Model regresi yang baik ialah model yang memiliki residu dan terdistribusi secara normal. Tes normalitas, tidak perlu dilakukan kepada setiap variabel yang ada, akan tetapi untuk nilai-nilai residual saja.

Uji normalitas dapat dilakukan dengan normal P-Plot, histogram, Chi square, tes kurtosis, skewness, kolmogorov- Smirnov, Shapiro-Wilk. Namun, uji normalitas tidak memiliki metode terbaik atau model paling tepat.

Dan saat ini kita menggunakan **Shapiro-Wilk**

In [19]:
test_stat, pvalue = shapiro(data.loc[data["experiment_group"] == "A", "invoice_created"])
print("p-value:",pvalue)
print("test_stat:",test_stat)

p-value: 0.0
test_stat: 0.6110898852348328


In [20]:
test_stat, pvalue = shapiro(data.loc[data["experiment_group"] == "B", "invoice_created"])
print("p-value:",pvalue)
print("test_stat:",test_stat)

p-value: 0.0
test_stat: 0.6110561490058899


Karena p-value dari `experiment_group` A & B adalah 0 maka asumsi normalitas tidak tersedia. (p-value < 0.05)


**b. Uji Homogenitas**

Uji homogenitas adalah suatu prosedur uji statistik yang bertujuan untuk memperlihatkan bahwa dua atau lebih kelompok data sampel yang telah diambil berasal dari populasi yang memiliki variansi yang sama. Dengan kata lain, uji homogenitas dilakukan untuk mengetahui bahwa himpunan data yang sedang diteliti memiliki karakteristik yang sama atau tidak.

Uji homogenitas varians suatu kelompok data dapat dilakukan dalam beberapa cara sebagai berikut uji F, uji Harley, uji Barlett, uji Levene dan uji Cochran.

Saat ini,  ada 2 kelompok data yang akan kita cari selisih masing-masing data dengan rata-rata kelompoknya maka kita akan menggunakan **Uji Levene**




In [21]:
test_stat, pvalue = levene(data.loc[data["experiment_group"] == "A", "invoice_created"],
                          data.loc[data["experiment_group"] == "B", "invoice_created"])
print("p-value:",pvalue)
print("test_stat:",test_stat)

p-value: 0.8900476735448392
test_stat: 0.01911139880146504


Karena p-value dari `experiment_group` A & B adalah 0.89 maka kelompok data memiliki varians yang sama (homogen). (p-value > 0.05)

**c. Uji Mann-Whitney**

Uji ini digunakan untuk mengetahui perbedaan median 2 kelompok bebas apabila skala data variabel terikatnya adalah ordinal atau interval/ratio tetapi tidak berdistribusi normal.

Karena pada *a. Uji Normalitas* disimpulkan bahwa normalitas tidak tersedia maka kita akan menggunakan **Uji Mann-Whitney** sebagai pengganti uji t.

In [22]:
test_stat, pvalue = mannwhitneyu(data.loc[data["experiment_group"] == "A", "invoice_created"],
                          data.loc[data["experiment_group"] == "B", "invoice_created"])

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 15922945908.0000, p-value = 0.8900


p-value(0.89) > 0.05

Maka kita gagal menolak **$H_0$** (harus menerima $H_0$) dimana conversion rate dari experiment keduanya tidak berbeda jauh.

---
Mari coba kita lihat jika conversion rate dibedakan dari `ui_version`. Sebelum kita lakukan perhitungan mencari conversion rate berdasarkan `ui_version`. Lakukan Uji Asumsi terlebih dahulu:

###**<font color = "green">Uji Asumsi `experiment_group` per `ui_version`**



In [23]:
# Uji Normalitas

test_stat_desktop_a, pvalue_desktop_a = shapiro(data.loc[(data["ui_version"] == "desktop") & (data["experiment_group"] == "A"), "invoice_created"])
test_stat_desktop_b, pvalue_desktop_b = shapiro(data.loc[(data["ui_version"] == "desktop") & (data["experiment_group"] == "B"), "invoice_created"])
test_stat_mobile_a, pvalue_mobile_a = shapiro(data.loc[(data["ui_version"] == "mobile") & (data["experiment_group"] == "A"), "invoice_created"])
test_stat_mobile_b, pvalue_mobile_b = shapiro(data.loc[(data["ui_version"] == "mobile") & (data["experiment_group"] == "B"), "invoice_created"])

print(f'p-value A in desktop = {pvalue_desktop_a}')
print(f'p-value B in desktop = {pvalue_desktop_b}')
print(f'p-value A in mobile = {pvalue_mobile_a}')
print(f'p-value B in mobile = {pvalue_mobile_b}')

p-value A in desktop = 0.0
p-value B in desktop = 0.0
p-value A in mobile = 0.0
p-value B in mobile = 0.0


p-value dari masing-masing group adalah 0 maka asumsi normalitas tidak tersedia. (p-value < 0.05)

In [24]:
test_stat, pvalue = levene(data.loc[(data["ui_version"] == "desktop") & (data["experiment_group"] == "A"), "invoice_created"],
                           data.loc[(data["ui_version"] == "mobile") & (data["experiment_group"] == "A"), "invoice_created"],
                           data.loc[(data["ui_version"] == "desktop") & (data["experiment_group"] == "B"), "invoice_created"],
                           data.loc[(data["ui_version"] == "mobile") & (data["experiment_group"] == "B"), "invoice_created"])

print("p-value:",pvalue)
print("test_stat:",test_stat)

p-value: 0.0
test_stat: 1004.8106664484541


p-value = 0 maka kelompok data tidak memiliki varians yang sama (tidak homogen). (p-value < 0.05)

**Kesimpulan :**
```Uji ini tidak bisa di lanjutkan```

---

###**<font color = "green">Analisa conversion `experiment_group` per `ui_version`**

In [25]:
# Select data with the conditions above
data_a_desktop = data[(data["experiment_group"] == "A") & (data["ui_version"] == "desktop")]
data_b_desktop = data[(data["experiment_group"] == "B") & (data["ui_version"] == "desktop")]
data_a_mobile = data[(data["experiment_group"] == "A") & (data["ui_version"] == "mobile")]
data_b_mobile = data[(data["experiment_group"] == "B") & (data["ui_version"] == "mobile")]

# display the data
data_a_desktop.head()

,internal_time,milliseconds,action,user_id,user_country,ui_version,experiment_group,num_event,invoice_created,id
6,12/19/2021 17:01,1.639920e+12,open-list,33026,KR,desktop,A,1,1,12/19/2021 17:0133026
7,12/15/2021 17:20,1.639580e+12,open-list,69463,HU,desktop,A,1,1,12/15/2021 17:2069463
11,12/17/2021 16:01,1.639750e+12,open-list,162121,BR,desktop,A,1,0,12/17/2021 16:01162121
12,12/17/2021 16:36,1.639750e+12,open-list,162121,BR,desktop,A,1,0,12/17/2021 16:36162121
13,12/17/2021 16:59,1.639750e+12,open-list,162121,BR,desktop,A,1,0,12/17/2021 16:59162121


In [26]:
data_b_desktop.head()

,internal_time,milliseconds,action,user_id,user_country,ui_version,experiment_group,num_event,invoice_created,id
8,12/19/2021 13:40,1.639910e+12,open-list,101545,HR,desktop,B,1,1,12/19/2021 13:40101545
9,12/20/2021 17:38,1.640010e+12,open-list,101545,HR,desktop,B,1,1,12/20/2021 17:38101545
10,12/20/2021 20:23,1.640020e+12,open-list,101545,HR,desktop,B,1,0,12/20/2021 20:23101545
14,12/14/2021 16:45,1.639490e+12,open-list,247542,FI,desktop,B,1,0,12/14/2021 16:45247542
15,12/18/2021 19:11,1.639840e+12,open-list,247542,FI,desktop,B,1,1,12/18/2021 19:11247542


In [27]:
data_a_mobile.head()

,internal_time,milliseconds,action,user_id,user_country,ui_version,experiment_group,num_event,invoice_created,id
166,12/10/2021 19:52,1.639160e+12,open-list,7896374,LT,mobile,A,1,1,12/10/2021 19:527896374
167,12/10/2021 21:59,1.639160e+12,open-list,7896374,LT,mobile,A,1,0,12/10/2021 21:597896374
1034,12/14/2021 18:28,1.639500e+12,open-list,43610997,FR,mobile,A,1,0,12/14/2021 18:2843610997
1296,12/17/2021 22:41,1.639770e+12,open-list,49998116,HR,mobile,A,1,1,12/17/2021 22:4149998116
1527,12/16/2021 22:07,1.639680e+12,open-list,56386438,BR,mobile,A,1,0,12/16/2021 22:0756386438


In [28]:
data_b_mobile.head()

,internal_time,milliseconds,action,user_id,user_country,ui_version,experiment_group,num_event,invoice_created,id
0,12/15/2021 4:10,1.639530e+12,open-list,2655,IL,mobile,B,1,1,12/15/2021 4:102655
1,12/15/2021 4:14,1.639530e+12,open-list,2655,IL,mobile,B,1,1,12/15/2021 4:142655
2,12/15/2021 4:15,1.639530e+12,open-list,2655,IL,mobile,B,1,0,12/15/2021 4:152655
3,12/15/2021 4:34,1.639530e+12,open-list,2655,IL,mobile,B,1,1,12/15/2021 4:342655
4,12/20/2021 8:27,1.639980e+12,open-list,2655,IL,mobile,B,1,1,12/20/2021 8:272655


In [29]:
# Number of users in the each group
n_a_desktop = data_a_desktop.shape[0]
n_b_desktop = data_b_desktop.shape[0]
n_a_mobile = data_a_mobile.shape[0]
n_b_mobile = data_b_mobile.shape[0]

print(f"Jumlah sample pada expreriment group A di Desktop ={n_a_desktop}")

print(f"Jumlah sample pada expreriment group B di Desktop ={n_b_desktop}")

print(f"Jumlah sample pada expreriment group A di Mobile ={n_a_mobile}")

print(f"Jumlah sample pada expreriment group B di Mobile ={n_b_mobile}")

Jumlah sample pada expreriment group A di Desktop =161282
Jumlah sample pada expreriment group B di Desktop =160783
Jumlah sample pada expreriment group A di Mobile =16863
Jumlah sample pada expreriment group B di Mobile =17941


Jelas data tidak homogen karena jumlah experiment setiap group berbeda sangat jauh. Perbedaan ini terlihat jelas antara `Desktop` dengan `Mobile`. Sekarang kita lihat sebanyak apa yg convert dari `Desktop` dan `Mobile`.

In [30]:
# Number of users convert in the each group
n_a_desktop_convert = data_a_desktop[data_a_desktop["invoice_created"] == 1].shape[0]
n_b_desktop_convert = data_b_desktop[data_b_desktop["invoice_created"] == 1].shape[0]
n_a_mobile_convert = data_a_mobile[data_a_mobile["invoice_created"] == 1].shape[0]
n_b_mobile_convert = data_b_mobile[data_b_mobile["invoice_created"] == 1].shape[0]

print(f"Jumlah sample pada expreriment group A di Desktop yg convert membuat invoice ={n_a_desktop_convert}")

print(f"Jumlah sample pada expreriment group B di Desktop yg convert membuat invoice ={n_b_desktop_convert}")

print(f"Jumlah sample pada expreriment group A di Mobile yg convert membuat invoice ={n_a_mobile_convert}")

print(f"Jumlah sample pada expreriment group B di Mobile yg convert membuat invoice ={n_b_mobile_convert}")

Jumlah sample pada expreriment group A di Desktop yg convert membuat invoice =61620
Jumlah sample pada expreriment group B di Desktop yg convert membuat invoice =61429
Jumlah sample pada expreriment group A di Mobile yg convert membuat invoice =3879
Jumlah sample pada expreriment group B di Mobile yg convert membuat invoice =4243


In [31]:
# Percentage in each group
persen_a_mobile = n_a_mobile_convert/n_a_mobile * 100
persen_a_desktop = n_a_desktop_convert/n_a_desktop * 100
persen_b_mobile = n_b_mobile_convert/n_b_mobile * 100
persen_b_desktop = n_b_desktop_convert/n_b_desktop * 100

print(f"% A di Desktop : {persen_a_desktop:.2f}%")
print(f"% B di Desktop : {persen_b_desktop:.2f}%")
print(f"% A di Mobile : {persen_a_mobile:.2f}%")
print(f"% B di Mobile : {persen_b_mobile:.2f}%")

% A di Desktop : 38.21%
% B di Desktop : 38.21%
% A di Mobile : 23.00%
% B di Mobile : 23.65%


**Kesimpulan :**
1. Id traffic dari Ads datang lebih banyak dari `Desktop`
2. Id convert to purchase lebih banyak dari `Desktop`
3. Tidak ada pengaruh dari gaya foto yg kita masukan didalam Ads.